In [1]:
import requests
import pandas as pd
import random
import json

from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from csv import DictReader

In [2]:
def scrape(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0'})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [3]:
def add_attribute(attribute):
    if attribute:
        return attribute.text.strip()
    return None

In [4]:
def convert_to_list(element):
    list = []

    for element in element:
        if element:
            return list.append(element.text.strip().replace('\n', ' change here'))
        return list.append(None)

In [5]:
def get_cookies_value(file):
    with open(file, encoding='utf-8-sig') as f:
        dict_reader = DictReader(f)
        list_of_dict = list(dict_reader)
    
    return list_of_dict

In [6]:
def get_product(url):
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    
    driver.get(url)

    cookies = get_cookies_value('/Users/alexandrecogordan/Documents/ESILV/Ongoing/Introduction to Webscrapping/Projet/cookies.csv')

    for cookie in cookies:
        driver.add_cookie(cookie)

    driver.refresh()

    button = driver.find_element(By.XPATH ,"//button[@class='a-headerMenuBtn icon-account layerTriggerAjax isAnonymous']")
    button.click()

    username = driver.find_element(By.XPATH, "//input[@id='username']")
    password = driver.find_element(By.XPATH, "//input[@id='password']")

    username.send_keys('alexandre.cogordan@icloud.com')
    password.send_keys('ninret-rynmu2-kermuQ')

    driver.implicitly_wait(5)

    # html = driver.page_source
    # soup = BeautifulSoup(html, features="html.parser")

    attributes = {'Name':[], 'Image Link':[], 'Origin':[], 'Price':[], 'Unit Price':[], 'Bio':[]}

    products = driver.find_elements(By.XPATH, "//div[@class='list__container']")

    for product in products:
        attributes['Name'].append(driver.find_elements(By.XPATH, "//p[@class='product-thumbnail__description"))
        attributes['Price'].append(driver.find_elements(By.XPATH, "//div[@class='product-price bolder text-dark-color"))
        attributes['Image Link'].append(driver.find_elements(By.XPATH, "//img[@class=product-thumbnail__image]"))
        attributes['Origin'].append(driver.find_elements(By.XPATH, "//span[@class=rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo]"))
        attributes['Unit Price'].append(driver.find_elements(By.XPATH, "//span[@class=text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy]"))

    driver.quit()

    # attributes['Name'] = convert_to_list(soup.find_all('p', class_='product-thumbnail__description'))
    # attributes['Price'] = convert_to_list(soup.find_all('div', class_='product-price bolder text-dark-color'))    
    # attributes['Image Link'] = convert_to_list(soup.find_all('img', class_='product-thumbnail__image'))
    # attributes['Origin'] = convert_to_list(soup.find_all('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo'))
    # attributes['Unit Price'] = convert_to_list(soup.find_all('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy'))

    for name in attributes['Name']:
        if name:
            if 'bio' in name.lower():
                attributes['Bio'].append(True)
            else:
                attributes['Bio'].append(False)
        else:
            attributes['Bio'].append(None)

    # for items in attributes:
    #     print(len(attributes[items]))

    attribute_df = pd.DataFrame(attributes) 
    attribute_df.dropna(how='all', inplace=True)

    return attribute_df

In [7]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [8]:
page_counter = 1
url = 'https://www.auchan.fr/fruits-legumes/fruits/ca-n0301?page=1'

while page_counter <= get_max_page(url):
    retail_df = get_product(url[:-1] + str(page_counter))
    page_counter += 1
    break

retail_df

KeyboardInterrupt: 

In [ ]:
#retail_df.to_csv('auchan.csv', index=False)